In [9]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras import regularizers

import os
import numpy as np
import pandas as pd

from sklearn import datasets, model_selection 
import matplotlib.pyplot as plt
%matplotlib inline

import tensorflow_hub as hub

In [10]:
def load_eurosat_data():
    x_train = np.load('data/eurosat/x_train.npy')
    y_train = np.load('data/eurosat/y_train.npy')
    x_test  = np.load('data/eurosat/x_test.npy')
    y_test  = np.load('data/eurosat/y_test.npy')
    return (x_train, y_train), (x_test, y_test)

(x_train, y_train), (x_test, y_test) = load_eurosat_data()
x_train = x_train / 255.0
x_test = x_test / 255.0

In [11]:
def get_new_model(input_shape):

    model = Sequential([
        Conv2D(filters=16, input_shape=input_shape, kernel_size=(3, 3), padding="same",
               activation='relu', name='conv_1'),
        Conv2D(filters=8, kernel_size=(3, 3), padding="same", activation='relu', name='conv_2'),
        MaxPooling2D(pool_size=(8, 8), name='pool_1'),
        Flatten(name='flatten'),
        Dense(units=32, activation='relu', name='dense_1'),
        Dense(units=10, activation='softmax', name='dense_2')
    ])
    model.compile(optimizer='adam',
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])
                  
    return model        

In [12]:
model = get_new_model(x_train[0].shape)

In [13]:
def get_test_accuracy(model, x_test, y_test):
    test_loss, test_acc = model.evaluate(x=x_test, y=y_test, verbose=0)
    print('accuracy: {acc:0.3f}'.format(acc=test_acc))

In [15]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv_1 (Conv2D)             (None, 64, 64, 16)        448       
                                                                 
 conv_2 (Conv2D)             (None, 64, 64, 8)         1160      
                                                                 
 pool_1 (MaxPooling2D)       (None, 8, 8, 8)           0         
                                                                 
 flatten (Flatten)           (None, 512)               0         
                                                                 
 dense_1 (Dense)             (None, 32)                16416     
                                                                 
 dense_2 (Dense)             (None, 10)                330       
                                                                 
Total params: 18,354
Trainable params: 18,354
Non-traina

In [16]:
get_test_accuracy(model, x_test, y_test)

accuracy: 0.095


In [17]:
def get_checkpoint_every_epoch():
    checkpoint_path = 'checkpoints_every_epoch/checkpoint_{epoch:03d}'
    checkpoint = ModelCheckpoint(filepath=checkpoint_path,
                                 frequency='epoch',
                                 save_weights_only=True,
                                 verbose=1)    
    return checkpoint   
    

def get_checkpoint_best_only():
    checkpoint_best_path = 'checkpoints_best_only/checkpoint'
    checkpoint_best = ModelCheckpoint(filepath=checkpoint_best_path,
                                  save_weights_only=True,
                                  save_freq='epoch',
                                  monitor='val_accuracy',
                                  save_best_only=True,
                                  verbose=1)
    return checkpoint_best  

In [18]:
def get_early_stopping():

    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=3)
    return early_stopping

In [19]:
checkpoint_every_epoch = get_checkpoint_every_epoch()
checkpoint_best_only = get_checkpoint_best_only()
early_stopping = get_early_stopping()

In [20]:
callbacks = [checkpoint_every_epoch, checkpoint_best_only, early_stopping]
model.fit(x_train, y_train, epochs=50, validation_data=(x_test, y_test), callbacks=callbacks)

Epoch 1/50
122/125 [============================>.] - ETA: 0s - loss: 1.9971 - accuracy: 0.2234
Epoch 1: saving model to checkpoints_every_epoch\checkpoint_001

Epoch 1: val_accuracy improved from -inf to 0.29200, saving model to checkpoints_best_only\checkpoint
125/125 [==============================] - 9s 63ms/step - loss: 1.9898 - accuracy: 0.2245 - val_loss: 1.7446 - val_accuracy: 0.2920
Epoch 2/50
121/125 [============================>.] - ETA: 0s - loss: 1.6572 - accuracy: 0.3525
Epoch 2: saving model to checkpoints_every_epoch\checkpoint_002

Epoch 2: val_accuracy improved from 0.29200 to 0.38000, saving model to checkpoints_best_only\checkpoint
125/125 [==============================] - 1s 9ms/step - loss: 1.6541 - accuracy: 0.3553 - val_loss: 1.5555 - val_accuracy: 0.3800
Epoch 3/50
125/125 [==============================] - ETA: 0s - loss: 1.4529 - accuracy: 0.4367
Epoch 3: saving model to checkpoints_every_epoch\checkpoint_003

Epoch 3: val_accuracy improved from 0.38000 to 

In [24]:
def get_model_last_epoch(model):

    checkpoint_path = 'checkpoints_every_epoch/checkpoint_018'
    model = model
    model.load_weights(checkpoint_path)
    
    return model 
    
    
def get_model_best_epoch(model):

    checkpoint_best_path = 'checkpoints_best_only/checkpoint'
    model = model
    model.load_weights(checkpoint_best_path)
    
    return model    

In [25]:
model1 = get_new_model(x_train[0].shape)

model_last_epoch = get_model_last_epoch(model1)
model_best_epoch = get_model_best_epoch(get_new_model(x_train[0].shape))

print('Model with last epoch weights:')
get_test_accuracy(model_last_epoch, x_test, y_test)
print('')
print('Model with best epoch weights:')
get_test_accuracy(model_best_epoch, x_test, y_test)

Model with last epoch weights:
accuracy: 0.679

Model with best epoch weights:
accuracy: 0.701


In [29]:
model_best_epoch.save('models/Week_4_best_model.h5')

In [30]:
def get_model_eurosatnet():

    model = load_model('models/Week_4_best_model.h5')
    
    return model  

In [31]:
model_eurosatnet = get_model_eurosatnet()
model_eurosatnet.summary()
get_test_accuracy(model_eurosatnet, x_test, y_test)

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv_1 (Conv2D)             (None, 64, 64, 16)        448       
                                                                 
 conv_2 (Conv2D)             (None, 64, 64, 8)         1160      
                                                                 
 pool_1 (MaxPooling2D)       (None, 8, 8, 8)           0         
                                                                 
 flatten (Flatten)           (None, 512)               0         
                                                                 
 dense_1 (Dense)             (None, 32)                16416     
                                                                 
 dense_2 (Dense)             (None, 10)                330       
                                                                 
Total params: 18,354
Trainable params: 18,354
Non-trai